This is a script for transferring PROF & STUDENT data from TXT to EXCEL while identifying known entities from the DigiKAR ontology lists.

First of all, we need to connect this Colab notebook with your Google Drive and define the directory for input and output data.


In [ ]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")
directory="/content/drive/My Drive/Colab_DigiKAR/"

In the second step, we have to install additional Packages needed for working with CSV, EXCEL and DataFrames.

In [ ]:
## install packages that are not part of Python's standard distribution

!pip install xlsxwriter
!pip install pandas
!pip install numpy

Now we can import the packages to the script and load our data.

In [ ]:
## import relevant packages

import xlsxwriter
import csv
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import re

The next step is to read the ontology files from either a directory or GITHUB. We read their content to a dataframe (table structure) that we can use to identify substrings in the input data.

In [ ]:
# define files containing ontological mapping

event_ontology='https://raw.githubusercontent.com/ieg-dhr/DigiKAR/main/OntologyFiles/event_ontology.csv' 
title_ontology='https://github.com/ieg-dhr/DigiKAR/blob/main/OntologyFiles/titles_ontology.csv?raw=true' 
function_ontology='https://github.com/ieg-dhr/DigiKAR/blob/main/OntologyFiles/functions_ontology.csv?raw=true'
place_ontology='https://raw.githubusercontent.com/ieg-dhr/DigiKAR/main/OntologyFiles/place_ontology.csv' 

# open ontology files and read content to dataframes

# READ EVENTS
data_e = pd.read_csv(event_ontology, sep=";")
events_old=data_e['event_old'].values.tolist()
    
# READ TITLES
data_t = pd.read_csv(title_ontology, sep=";")
title_old=data_t['title_old'].values.tolist()

# READ FUNCTIONS
data_f = pd.read_csv(function_ontology, sep=";")
function_old=data_f['function_old'].values.tolist()

# READ PLACES
data_p = pd.read_csv(place_ontology, sep=";")
places_old=data_p['place_old'].values.tolist()


Now we define and execute the function that reads information from a semi-structured TXT file with out OCR data to another data frame. The preliminary columns we assign are "name", "info" and "source". Then we split up the "info" into several individual events. For this process, we use the split() function in Python and several delimiters: #PERSON, #SOURCE and the semi-colon ";".

In [ ]:
# function to process data
def extract_information(filenames):
        
# read files in directory to EXCEL
    name_list=[]
    source_list=[]
    info_list=[]
    for item in os.listdir(filenames):
        # using readlines()
        with open(os.path.join(filenames, item)) as f:
          contents=f.read()
          #print(contents[:500])
          # split on hashtag to extract entries for each person
          persons=contents.split("#PERSON")
          # check substrings
          for p in persons:
            source=p.split("#SOURCE")
            try:
              p_new=source[0]
              s_new=source[1]
          # separate name from events
              events=p_new.split(';')
              n_items=len(events)
              name_p=[events[0]] * (n_items-1)
              source_p=[s_new] * (n_items-1)
              for n in name_p:
                name_list.append(n)
              for s in source_p:
                source_list.append(s)
              for e in events[1:(n_items)]:
                #print("ITEM: ", events[0], ":", e)
                info_list.append(e)
            except IndexError:
              continue

          # add data to new dataframe

          global df_size
          df_size = len(info_list)
          #print(len(name_list)) # to check that all lists have the same length
          #print(len(source_list)) # to check that all lists have the same length
          
          data = {
              'Name': name_list,
              'Info': info_list,
              'Source': source_list
                }
          global event_df
          event_df = pd.DataFrame(data)

# show data frame if small enough
    try:
        display(event_df) 
    except:
        display(event_df[:100]) # only show first 100 rows
          
# call function with input

filenames = directory+"ReadTXTtoEXCEL_profs"
extract_information(filenames)


The above section of the script creates a link between each name and individual events associated with that name, e.g. 

```
NAME: ZURMÜHLEN, Paulus Josephus Ignatius
EVENT: cand. jur., 24.9.1751
```

Our next script section attempts to identify known entities in the event string and write relevant information to "event", "title" or "function" columns in the existing data frame. This is difficult because a lot of Latin expressions and abbreviations in both Latin and German have been used.

In [ ]:
from traitlets.config.application import T
# Check INFO column in existing DF
place_list=[]
title_list=[]
event_list=[]
function_list=[]
date_list=[]

try:
  for x in range(0, df_size):
      print(df_size - x)
      e_df=event_df.iloc[[x]]
      event_x_list=[]
      title_x_list=[]
      place_x_list=[]
      function_x_list=[]
      date_x_list=[]

    # FIND DATE
      d_values=e_df["Info"].values[0]
      print(d_values)
      try:
          x_date = re.findall("[\d]+[.][\d]+[.][\d]+ | [1-3][0-9]{3}", d_values) # DD.MM.YYYY or YYYY
          for x in x_date:
            if 6>len(x)>4:
                date_x_list.append(x.strip())
            else:
                date_in=x.split(".")
                print(date_in)
                year=str(date_in[2].strip())
                if len(date_in[1])>1:
                  month=str(date_in[1])
                else:
                  month=str("0"+date_in[1])
                if len(date_in[0])>1:
                  day=str(date_in[0])
                else:
                  day=str("0"+date_in[0])
                date_out=str(year + "-" + month + "-" + day)
                print(date_out)
                date_x_list.append(date_out)
      except:
          date_x_list.append("#")
      date_list.append(date_x_list)

    # FIND EVENT       
      for e_old in events_old: 
          e_new=data_e.loc[data_e['event_old'] == e_old, 'event_type'].values[0]
          e_values=e_df["Info"].values[0]
          if e_old in e_values:
              #print("MATCH EVENT: ", e_new)
              event_x_list.append(e_new)
          else:
              event_x_list.append("#")
      event_list.append(max(event_x_list, key=len))

    # FIND TITLE
      for t_old in title_old:
          t_values=e_df["Info"].values[0]
          if t_old in t_values:
              #print("MATCH TITLE: ", t_old)
              title_x_list.append(t_old)
          else:
              title_x_list.append("#")
      title_list.append(max(title_x_list, key=len))

    # FIND PLACES
      for p_old in places_old:
          p_new=data_p.loc[data_p['place_old'] == p_old, 'place_new'].values[0]
          if p_old in e_df["Info"].values[0]:
              print("MATCH PLACE: ", p_old)
              place_x_list.append(p_new)
          else:
              place_x_list.append("#")
      place_list.append(set(place_x_list))
                
    # FIND FUNCTION
      for f_old in function_old:
          if f_old in e_df["Info"].values[0]:
              #print("MATCH FUNCTION: ", f_old)
              function_x_list.append(f_old)
          else:
              function_x_list.append("#")
      function_list.append(max(function_x_list, key=len))

except IndexError:
  print("No more data found.")



# add new data to existing data frame
event_df["events"] = event_list
event_df["places"] = place_list
event_df["titles"] = title_list
event_df["functions"] = function_list
try:
  event_df["date"] = date_list # add flattened list: [item for sublist in date_list for item in sublist]
except ValueError:
  print(date_list)

display(event_df[:100])

# write all results to new EXCEL file

workbook=directory+'Profs_OCR_factoid_with-dates.xlsx'
writer = pd.ExcelWriter(workbook, engine='xlsxwriter') # create a Pandas Excel writer using XlsxWriter as the engine.
event_df.to_excel(writer, sheet_name='OCR-to-factoid') # Convert the dataframe to an XlsxWriter Excel object.
writer.save() # Close the Pandas Excel writer and output the Excel file.

Matches found in the text are displayed in the "MATCH: WORD" format to help you track the live performance of the script.


Check the output files and repeat process with refined ontology files if necessary.

Script by Monika Barget, Maastricht/Mainz

January 2023
